### Metadata extrahieren 
Von allen 9 811 Texten Metadaten einlesen und als Dataframe speichern 

In [ ]:
import glob, os
from pathlib import Path
import html
from tokenize import tokenize

In [ ]:
from unidecode import unidecode
from urllib.parse import unquote
import pandas as pd
import re

In [ ]:
# alle Harry Potter Fan-Fiction Texte werden eingelesen
p = Path('./Fan-Fiction/Harry Potter')
files = list(p.glob('*'))

In [ ]:
len(files)

Datum herausfiltern und in pd.Series speichern/ Index = Dateiname

In [ ]:
def get_datum_series(text, index):
    # input: text
    # get Series of Datum
    import re
    datum = re.findall('<date>(.+?)</date>', text)
    datumJoined = " ".join(datum)
    datum_series = pd.Series(datumJoined, index=[index])
    return datum_series

In [ ]:
def get_title_series(text, index):
    # input: text
    # get Series of Titel
    import re
    text2 = unidecode(unquote(text))
    title = re.findall('<title type="main">(.+?)</title>', text2)
    titleJoined = " ".join(title)
    title_series = pd.Series(titleJoined, index=[index])
    return title_series

In [ ]:
def get_author_series(text: object, index: object) -> object:
    # input: text
    # get Series of Autor
    import re
    author = re.findall('<persName>(.+?)</persName>', text)
    authorJoined = " ".join(author)
    author_series = pd.Series(authorJoined, index=[index])
    return author_series

In [ ]:
def get_story_series(text, index):
    # input: text
    # get Series of Story
    import re
    text2 = html.unescape(text)
    story = re.findall('<body>(.+?)</body>', text2)

    # Text von Tags bereinigen
    counter = 0
    stories = []
    for f in story:
        new_story = re.sub('<[^<]+>', "", f)

        counter += 1
        stories.append(new_story)

    storyJoined = " ".join(stories)
    story_series = pd.Series(storyJoined, index=[index])
    return story_series

In [ ]:
counter = 0
datum_series = pd.Series(dtype="Int64")
title_series = pd.Series(dtype="string")
author_series = pd.Series(dtype="string")
story_series = pd.Series(dtype="string")
for file in files:
    with open(file, encoding="utf-8") as fin:
        text = fin.read()
        file = os.path.basename(file)

        datumSeries = get_datum_series(text, file)
        datum_series = datum_series.append(datumSeries)

        titleSeries = get_title_series(text, file)
        title_series = title_series.append(titleSeries)

        authorSeries = get_author_series(text, file)
        author_series = author_series.append(authorSeries)

        storySeries = get_story_series(text, file)
        story_series = story_series.append(storySeries)

        counter += 1

In [ ]:
# Series in Dataframe packen; Index= Dateiname
df = pd.DataFrame({'Autor': author_series, 'Titel': title_series, 'Datum': datum_series, "Text": story_series})
# Dataframe zu CSV-Datei
df.to_csv('Df_one.csv', encoding='utf-8')

In [ ]:
df.head()

Originaltexte einlesen 

In [16]:
import glob, os
import shutil
from pathlib import Path


# alle Original Texte werden eingelesen
p = Path('./Original')
original_files = list(p.glob('*'))

In [17]:
original_files

[WindowsPath('Original/01_Harry Potter und der Gefangene - J.K. Rowling.txt'),
 WindowsPath('Original/02_Harry Potter und der Halbblutpr - J.K. Rowling.txt'),
 WindowsPath('Original/03_Harry Potter und der Orden des - J.K. Rowling.txt'),
 WindowsPath('Original/04_Harry Potter und der Stein der - J.K. Rowling.txt'),
 WindowsPath('Original/05_Harry Potter und die Heiligtume - J.K. Rowling.txt'),
 WindowsPath('Original/06_Harry Potter und die Kammer des - J.K. Rowling.txt'),
 WindowsPath('Original/07_Harry Potter und der Feuerkelch - J.K. Rowling.txt')]

In [18]:
#den Inhalt der Texte einlesen und in Liste speichern

counter = 0
my_original_files = []
for file in original_files:
    with open(file, encoding="utf-8") as fin:
        text = fin.read()
        file = os.path.basename(file)
        counter += 1
        my_original_files.append(text)

In [19]:
len(my_original_files)

7

In [20]:
df2 = pd.DataFrame({'Text':my_original_files})

In [21]:
df2

,Text
0,"\n\n\nFür Jill Prewett und Aine Kiely,\n\ndie ..."
1,"\n\n\nMackenzie,\n\nmeiner schönen Tochter,\n\..."
2,"\n\n\n\n\nFür Neil, Jessica und David,\n\ndie ..."
3,"\n\n\n\nFür Jessica, für Anne und fü Di;\n\nJe..."
4,\n\n\nDie\n\nWidmung\n\ndieses Buches\n\nist\n...
5,"\n\n\nFür Sean P. F. Harris,\n\nFluchtwagen-Fa..."
6,"Für Peter Rowling,\n\nin Erinnerung an Mr Ridl..."


Original_Texte laden 

In [22]:
import pandas as pd

df = pd.read_excel('df_original.xlsx', encoding='utf-8')

In [23]:
df.to_excel("df_original.xlsx")

In [24]:
df["Text"] = df2.Text

In [25]:
df

,Unnamed: 0,Autor,Titel,Übersetzer,Original_Titel,ISBN,Datum,Text
0,0,J.K. Rowling,HARRY POTTER und der Gefangene von Askaban,Klaus Fritz,Harry Potter and the Prisoner of Azkaban,978-1-78110-057-8,1999,"\n\n\nFür Jill Prewett und Aine Kiely,\n\ndie ..."
1,1,J.K. Rowling,HARRY POTTER und der Halbblutprinz,Klaus Fritz,Harry Potter and the Half-Blood Prince,978-1-78110-060-8,2005,"\n\n\nMackenzie,\n\nmeiner schönen Tochter,\n\..."
2,2,J.K. Rowling,HARRY POTTER und der Orden des Phönix,Klaus Fritz,Harry Potter and the Order of the Phoenix,978-1-78110-059-2,2003,"\n\n\n\n\nFür Neil, Jessica und David,\n\ndie ..."
3,3,J.K. Rowling,HARRY POTTER und der Stein der Weisen,Klaus Fritz,Harry Potter and the Philosopher’s Stone,978-1-78110-055-4,1997,"\n\n\n\nFür Jessica, für Anne und fü Di;\n\nJe..."
4,4,J.K. Rowling,HARRY POTTER und die Heiligtümer des Todes,Klaus Fritz,Harry Potter and the Deathly Hallows,978-1-78110-061-5,2007,\n\n\nDie\n\nWidmung\n\ndieses Buches\n\nist\n...
5,5,J.K. Rowling,HARRY POTTER und die Kammer des Schreckens,Klaus Fritz,Harry Potter and the Chamber of Secrets,978-1-78110-056-1,1998,"\n\n\nFür Sean P. F. Harris,\n\nFluchtwagen-Fa..."
6,6,J.K. Rowling,HARRY POTTER und der Feuerkelch,Klaus Fritz,Harry Potter and the Goblet of Fire,978-1-78110-058-5,2000,"Für Peter Rowling,\n\nin Erinnerung an Mr Ridl..."


Stoppwörter aus den Texten entfernen und als neue Spalte speichern

In [26]:
df['removed_stop_word_lower'] = df['Text'].str.lower()

In [27]:
# stopwords entfernen 
import nltk
from nltk.corpus import stopwords
german_stop_words = stopwords.words('german')

def stop_word_removal(x):
    token = x.split()
    return ' '.join([w for w in token if not w in german_stop_words])


df['removed_stop_word']  = df['removed_stop_word_lower'].apply(stop_word_removal)


In [28]:
df

,Unnamed: 0,Autor,Titel,Übersetzer,Original_Titel,ISBN,Datum,Text,removed_stop_word_lower,removed_stop_word
0,0,J.K. Rowling,HARRY POTTER und der Gefangene von Askaban,Klaus Fritz,Harry Potter and the Prisoner of Azkaban,978-1-78110-057-8,1999,"\n\n\nFür Jill Prewett und Aine Kiely,\n\ndie ...","\n\n\nfür jill prewett und aine kiely,\n\ndie ...","jill prewett aine kiely, patinnen souls inhalt..."
1,1,J.K. Rowling,HARRY POTTER und der Halbblutprinz,Klaus Fritz,Harry Potter and the Half-Blood Prince,978-1-78110-060-8,2005,"\n\n\nMackenzie,\n\nmeiner schönen Tochter,\n\...","\n\n\nmackenzie,\n\nmeiner schönen tochter,\n\...","mackenzie, schönen tochter, widme zwilling tin..."
2,2,J.K. Rowling,HARRY POTTER und der Orden des Phönix,Klaus Fritz,Harry Potter and the Order of the Phoenix,978-1-78110-059-2,2003,"\n\n\n\n\nFür Neil, Jessica und David,\n\ndie ...","\n\n\n\n\nfür neil, jessica und david,\n\ndie ...","neil, jessica david, welt verzaubern inhalt du..."
3,3,J.K. Rowling,HARRY POTTER und der Stein der Weisen,Klaus Fritz,Harry Potter and the Philosopher’s Stone,978-1-78110-055-4,1997,"\n\n\n\nFür Jessica, für Anne und fü Di;\n\nJe...","\n\n\n\nfür jessica, für anne und fü di;\n\nje...","jessica, anne fü di; jessica mag geschichten, ..."
4,4,J.K. Rowling,HARRY POTTER und die Heiligtümer des Todes,Klaus Fritz,Harry Potter and the Deathly Hallows,978-1-78110-061-5,2007,\n\n\nDie\n\nWidmung\n\ndieses Buches\n\nist\n...,\n\n\ndie\n\nwidmung\n\ndieses buches\n\nist\n...,"widmung buches siebengeteilt: neil, jessica, d..."
5,5,J.K. Rowling,HARRY POTTER und die Kammer des Schreckens,Klaus Fritz,Harry Potter and the Chamber of Secrets,978-1-78110-056-1,1998,"\n\n\nFür Sean P. F. Harris,\n\nFluchtwagen-Fa...","\n\n\nfür sean p. f. harris,\n\nfluchtwagen-fa...","sean p. f. harris, fluchtwagen-fahrer freund s..."
6,6,J.K. Rowling,HARRY POTTER und der Feuerkelch,Klaus Fritz,Harry Potter and the Goblet of Fire,978-1-78110-058-5,2000,"Für Peter Rowling,\n\nin Erinnerung an Mr Ridl...","für peter rowling,\n\nin erinnerung an mr ridl...","peter rowling, erinnerung mr ridley, susan sla..."


In [30]:
df.to_csv('df_original.csv', encoding='utf-8')